# Data Science

- Student: Dmytro Geleshko
- Group: IP-91
- Var: 6

## Imports

In [129]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Config

In [119]:
# var 6
np.random.seed(6)
plt.rcParams["figure.figsize"] = (20, 8)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.3f}".format

## Functions

In [120]:
def lsm_create_f(size, pol_pow):
    """
    Creating F matrix:
    1, x, x^2, ..., x^PARAM_max_pow
    """
    
    F = np.ones((size, pol_pow + 1))
    for i in range(1, pol_pow + 1):
        # pow
        F[:, i] = np.power(np.arange(0, size), i)
    return F


def lsm_coef(y, F):
    """
    Returns matrix of coef. C.
    """

    # 1-d array to row vector
    Y = y.reshape(-1, 1)
    # calc
    FT = F.T
    FF = np.dot(FT, F)
    FFI = np.linalg.inv(FF)
    FFIFT = np.dot(FFI, FT)
    # return coef
    return np.dot(FFIFT, Y)


def lsm(y, pol_pow):
    """
    LSM
    y - 1d array of values,
    pol_pow - polynomial power of model
    """

    F = lsm_create_f(len(y), pol_pow)
    C = lsm_coef(y, F)
    # returning array, reshaped from row vector to 1d
    return np.dot(F, C).reshape(-1)

In [121]:
def find_anomalies(data, sliding_window_size, lsm_pow, std3):
    """
    Returns indexes of anomalies.
    """
    anomaly_index = []
    
    for i in range(0, len(data) - sliding_window_size):
        # new window
        window = data[i:(sliding_window_size + i)]
        # finding C matrix for current window
        coef = lsm_coef(window, lsm_create_f(sliding_window_size, lsm_pow))
        # prediction of next value
        y_hat = np.dot(lsm_create_f(sliding_window_size + 1, lsm_pow), coef).reshape(-1)[-1]
        # if difference between predicted and real value is more than 3*std => anomaly
        if abs(data[sliding_window_size + i] - y_hat) > std3:
            anomaly_index.append(sliding_window_size + i)
    return anomaly_index

In [122]:
def mse(data1, data2):
    return (np.square(data1 - data2)).mean()

# Data

## Load And Format Data

In [123]:
df = pd.read_excel("Data_Set_6.xlsx", index_col=[0, 1], thousands=",")

# replace missing values
df.replace("not avilable", np.NaN, inplace=True)
df.replace("n.a.", np.NaN, inplace=True)
df.replace(-1, np.NaN, inplace=True)

# lower the names of columns and indexes
df.columns = [name.lower() for name in df.columns]
df.index.set_names([name.lower() for name in df.index.names], inplace=True)


# sort by sale_id
df.sort_index(inplace=True)
df

january  february     march     april   may  \
sales_id sales_by_region                                                 
1        AUH             3,469.000       NaN       NaN 3,642.000  5803   
         SHJ             5,840.000 5,270.000 4,114.000 5,605.000  4387   
         UAQ             2,967.000 2,425.000 5,353.000 3,547.000  5027   
2        AUH             1,328.000 4,264.000 1,574.000 2,343.000  3826   
3        AUH             1,722.000   956.000 1,297.000 1,984.000  2744   
...                            ...       ...       ...       ...   ...   
29       UAQ             5,325.000 1,905.000 5,049.000 1,311.000  4146   
30        FUJ            3,402.000 5,283.000 2,229.000 3,758.000  1427   
          FUJ            5,549.000 1,302.000 1,929.000 2,822.000  5379   
         AJM             2,832.000 5,978.000 1,684.000 1,550.000  1194   
         AJM             2,028.000 2,006.000 5,120.000 5,959.000  3127   

                              june      july    august  september   october  \
sales_id sales_by_region                                                      
1        AUH             5,662.000 1,896.000 2,293.000       2583 5,233.000   
         SHJ             5,026.000 4,055.000 2,782.000       4578 4,993.000   
         UAQ             4,078.000 3,858.000 1,927.000       3527 4,179.000   
2        AUH             4,932.000 1,710.000 3,221.000       3381 1,313.000   
3        AUH             5,793.000 2,261.000 5,607.000       2437 4,328.000   
...                            ...       ...       ...        ...       ...   
29       UAQ             1,706.000 1,689.000 3,190.000       2915 2,183.000   
30        FUJ            1,057.000 5,277.000 5,231.000       3909 4,345.000   
          FUJ            1,243.000 3,075.000 4,358.000       5106 2,322.000   
         AJM             3,737.000 5,779.000 4,441.000       1213 3,711.000   
         AJM             3,962.000 4,780.000 3,200.000       1836 2,623.000   

                          november  december  
sales_id sales_by_region                      
1        AUH                  4421 4,071.000  
         SHJ                  2859 4,853.000  
         UAQ                  1571 5,551.000  
2        AUH                  1765 1,214.000  
3        AUH                  3317 5,390.000  
...                            ...       ...  
29       UAQ                  3301 4,365.000  
30        FUJ                 5287 2,638.000  
          FUJ                 2409 1,069.000  
         AJM                  5384 1,293.000  
         AJM                  1607 2,371.000  

[98 rows x 12 columns]

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 98 entries, (1, 'AUH') to (30, 'AJM')
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   january    92 non-null     float64
 1   february   96 non-null     float64
 2   march      95 non-null     float64
 3   april      97 non-null     float64
 4   may        98 non-null     int64  
 5   june       95 non-null     float64
 6   july       97 non-null     float64
 7   august     96 non-null     float64
 8   september  98 non-null     int64  
 9   october    97 non-null     float64
 10  november   98 non-null     int64  
 11  december   96 non-null     float64
dtypes: float64(9), int64(3)
memory usage: 11.1+ KB


## Fill NaN

In [125]:
df = df.ffill(axis=1).bfill(axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 98 entries, (1, 'AUH') to (30, 'AJM')
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   january    98 non-null     float64
 1   february   98 non-null     float64
 2   march      98 non-null     float64
 3   april      98 non-null     float64
 4   may        98 non-null     float64
 5   june       98 non-null     float64
 6   july       98 non-null     float64
 7   august     98 non-null     float64
 8   september  98 non-null     float64
 9   october    98 non-null     float64
 10  november   98 non-null     float64
 11  december   98 non-null     float64
dtypes: float64(12)
memory usage: 11.1+ KB


## Cast to np.float32

In [127]:
df[df.columns] = df[df.columns].values.astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 98 entries, (1, 'AUH') to (30, 'AJM')
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   january    98 non-null     float32
 1   february   98 non-null     float32
 2   march      98 non-null     float32
 3   april      98 non-null     float32
 4   may        98 non-null     float32
 5   june       98 non-null     float32
 6   july       98 non-null     float32
 7   august     98 non-null     float32
 8   september  98 non-null     float32
 9   october    98 non-null     float32
 10  november   98 non-null     float32
 11  december   98 non-null     float32
dtypes: float32(12)
memory usage: 6.5+ KB


In [128]:
df

january  february     march     april       may  \
sales_id sales_by_region                                                     
1        AUH             3,469.000 3,469.000 3,469.000 3,642.000 5,803.000   
         SHJ             5,840.000 5,270.000 4,114.000 5,605.000 4,387.000   
         UAQ             2,967.000 2,425.000 5,353.000 3,547.000 5,027.000   
2        AUH             1,328.000 4,264.000 1,574.000 2,343.000 3,826.000   
3        AUH             1,722.000   956.000 1,297.000 1,984.000 2,744.000   
...                            ...       ...       ...       ...       ...   
29       UAQ             5,325.000 1,905.000 5,049.000 1,311.000 4,146.000   
30        FUJ            3,402.000 5,283.000 2,229.000 3,758.000 1,427.000   
          FUJ            5,549.000 1,302.000 1,929.000 2,822.000 5,379.000   
         AJM             2,832.000 5,978.000 1,684.000 1,550.000 1,194.000   
         AJM             2,028.000 2,006.000 5,120.000 5,959.000 3,127.000   

                              june      july    august  september   october  \
sales_id sales_by_region                                                      
1        AUH             5,662.000 1,896.000 2,293.000  2,583.000 5,233.000   
         SHJ             5,026.000 4,055.000 2,782.000  4,578.000 4,993.000   
         UAQ             4,078.000 3,858.000 1,927.000  3,527.000 4,179.000   
2        AUH             4,932.000 1,710.000 3,221.000  3,381.000 1,313.000   
3        AUH             5,793.000 2,261.000 5,607.000  2,437.000 4,328.000   
...                            ...       ...       ...        ...       ...   
29       UAQ             1,706.000 1,689.000 3,190.000  2,915.000 2,183.000   
30        FUJ            1,057.000 5,277.000 5,231.000  3,909.000 4,345.000   
          FUJ            1,243.000 3,075.000 4,358.000  5,106.000 2,322.000   
         AJM             3,737.000 5,779.000 4,441.000  1,213.000 3,711.000   
         AJM             3,962.000 4,780.000 3,200.000  1,836.000 2,623.000   

                          november  december  
sales_id sales_by_region                      
1        AUH             4,421.000 4,071.000  
         SHJ             2,859.000 4,853.000  
         UAQ             1,571.000 5,551.000  
2        AUH             1,765.000 1,214.000  
3        AUH             3,317.000 5,390.000  
...                            ...       ...  
29       UAQ             3,301.000 4,365.000  
30        FUJ            5,287.000 2,638.000  
          FUJ            2,409.000 1,069.000  
         AJM             5,384.000 1,293.000  
         AJM             1,607.000 2,371.000  

[98 rows x 12 columns]

## Finding optimal polynomial power for smoothing with lsm

errs = []
pow_range = range(1, PARAM_max_test_pow)
for power in pow_range:
    errs.append((np.square(data["quadratic_normal_anomalies_fix"] - lsm(data["quadratic_normal_anomalies_fix"], power))).mean())

plt.title("MSE")
plt.xlabel("Polynomial power")
plt.plot(pow_range, errs)
plt.show()